## Download data set from 
https://www.kaggle.com/datasets/alechantson/carladataset?resource=download
###  The downloaded Images are kept in the same folder with names train and valid 
### Rearranging the images and yaml file to train a custom detection model using yolov8n

In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
import cv2

In [2]:
#Load all txt files and store in a list
train_txt_list = glob('train/labels/*.txt')
valid_txt_list = glob('valid/labels/*.txt')

#replace \\ with /
train_txt_list = list(map(lambda x: x.replace('\\','/'), train_txt_list))
valid_txt_list = list(map(lambda x: x.replace('\\','/'), valid_txt_list))

In [3]:
print(len(train_txt_list),len(valid_txt_list))

1600 264


#### The given data has 10 classes names: 
#### bike, motobike, person, traffic_light_green, traffic_light_orange, traffic_light_red, traffic_sign_30, traffic_sign_60, traffic_sign_90, vehicle
#### Filtering the images and restrciting to only 2 classes to train the model : Bike and Vehicle. by merging both bike and motorbike.

In [4]:
def txt_to_list(txt_list):
    
    yolo_labels_list = []
    for txt_file in txt_list:

        with open(txt_file, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            line = line.strip().split(' ')
            label = line[0]
            
            label_list = [0,1,9]
            
            x_center, y_center, width, height = line[1:]
        

            txt_name = txt_file.split('/')[2]
            img_name = txt_name.split('.txt') 
            name = img_name[0] +'.jpg'
            
            if int(label) in label_list:
                
                if int(label) == 9:
                    label = '1'
                elif int(label) == 1:
                    label = '0'
                
                yolo_label = [name, label, x_center, y_center, width, height]
                yolo_labels_list.append(yolo_label)
    
    return yolo_labels_list

yolo_train_list = txt_to_list(train_txt_list)
yolo_valid_list = txt_to_list(valid_txt_list)

print(len(yolo_train_list),len(yolo_valid_list))

1322 187


## Splitting the data into testing data and Training data

In [5]:
train_df = pd.DataFrame(yolo_train_list, columns= ['filename','id','x_center', 'y_center', 'width', 'height'])
valid_df = pd.DataFrame(yolo_valid_list, columns= ['filename','id','x_center', 'y_center', 'width', 'height'])

In [6]:
train_df

,filename,id,x_center,y_center,width,height
0,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,1,0.57421875,0.7039473684210527,0.2796875,0.35526315789473684
1,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,1,0.27421875,0.5328947368421053,0.1078125,0.05
2,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,1,0.86796875,0.5236842105263158,0.1046875,0.05263157894736842
3,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,1,0.5796875,0.5,0.0375,0.031578947368421054
4,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,0,0.6046875,0.5039473684210526,0.01875,0.05
...,...,...,...,...,...,...
1317,998_jpg.rf.8eeb8ce8d07dd2a1c366a1041b79794c.jpg,1,0.33515625,0.55625,0.1234375,0.07916666666666666
1318,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,1,0.14453125,0.471875,0.2828125,0.8604166666666667
1319,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,1,0.92265625,0.528125,0.1515625,0.09375
1320,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,1,0.5,0.5322916666666667,0.053125,0.05625


In [7]:
valid_df

,filename,id,x_center,y_center,width,height
0,103_jpg.rf.f3b0bc74d3bdd7b844d404aa5ba65e08.jpg,1,0.51171875,0.5364583333333334,0.0609375,0.06041666666666667
1,103_jpg.rf.f3b0bc74d3bdd7b844d404aa5ba65e08.jpg,1,0.38984375,0.540625,0.0515625,0.06875
2,103_jpg.rf.f3b0bc74d3bdd7b844d404aa5ba65e08.jpg,1,0.325,0.5697916666666667,0.125,0.11458333333333333
3,103_jpg.rf.f3b0bc74d3bdd7b844d404aa5ba65e08.jpg,1,0.14453125,0.6895833333333333,0.2890625,0.37083333333333335
4,105_jpg.rf.30798cd774fb9e7f5dd2ec49fbfaceee.jpg,1,0.44375,0.5210526315789473,0.028125,0.03684210526315789
...,...,...,...,...,...,...
182,94_jpg.rf.fffa7f9eff1ccc1b918f482fdf3b5f18.jpg,1,0.3984375,0.5236842105263158,0.0375,0.03684210526315789
183,95_jpg.rf.76416cde0656b250a676f4d0dc983707.jpg,1,0.4984375,0.7631578947368421,0.38125,0.4631578947368421
184,96_jpg.rf.021fb42c41c3b9d41e352e7db8458bd4.jpg,1,0.9375,0.55,0.121875,0.06842105263157895
185,98_jpg.rf.fb37528f11f2233af05905d0bb44b34d.jpg,1,0.58671875,0.506578947368421,0.0328125,0.02894736842105263


In [8]:
train_images = train_df['filename'].unique()
valid_images = valid_df['filename'].unique()
print(len(train_images),len(valid_images))

772 106


In [9]:
import os
from shutil import copy

## Create folders as mentioned in the working directory

In [10]:
train_folder = 'train_images/train'
valid_folder = 'train_images/valid'

In [11]:
groupby_obj_train = train_df.groupby('filename')
groupby_obj_valid = valid_df.groupby('filename')

In [12]:
def save_data(filename, src_path, folder_path, group_obj):
    # move image
    src = os.path.join(src_path,filename)
    dst = os.path.join(folder_path,filename)
    copy(src,dst) # move image to the destination folder
    
    # save the labels
    text_filename = os.path.join(folder_path,
                                 os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)

# Mention the source images and labels

In [13]:
train_src_path = 'train/images'
valid_src_path = 'valid/images'

filename_series_train = pd.Series(groupby_obj_train.groups.keys())
filename_series_valid = pd.Series(groupby_obj_valid.groups.keys())

In [14]:
filename_series_train.apply(save_data,args=(train_src_path, train_folder,groupby_obj_train))

0      None
1      None
2      None
3      None
4      None
       ... 
767    None
768    None
769    None
770    None
771    None
Length: 772, dtype: object

In [15]:
filename_series_valid.apply(save_data,args=(valid_src_path, valid_folder,groupby_obj_valid))

0      None
1      None
2      None
3      None
4      None
       ... 
101    None
102    None
103    None
104    None
105    None
Length: 106, dtype: object